## Part 1. Forgetful ChatBot

In [70]:
import langchain
from pprint import pprint
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [71]:
# make sure OPENAI_API_KEY environment variable is defined
chatGPT = ChatOpenAI()
langchain.debug = False

In [72]:
messages = [
    HumanMessage(content="What is the population of the German capital Berlin?")
]
print(chatGPT(messages))

content='As of 2021, the estimated population of Berlin is approximately 3.8 million people.' additional_kwargs={} example=False


In [75]:
messages = [
    HumanMessage(content="How does it compare to the population of London?")
]
print(chatGPT(messages))

content="To provide an accurate comparison, we need to know the specific population you are referring to. As of 2021, the population of Greater London is estimated to be around 9 million people. Please specify the population you want to compare to London's population for a more precise answer." additional_kwargs={} example=False


## Part 2. + memory

In [80]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.save_context(
    {"input": "What is the population of the German capital Berlin?"}, 
    {"output": "As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million."}
)

In [81]:
print(memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What is the population of the German capital Berlin?', additional_kwargs={}, example=False), AIMessage(content='As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'


In [82]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [83]:
# make sure OPENAI_API_KEY environment variable is defined
chatGPT = ChatOpenAI()

template = """Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
memory.save_context(
    {"input": "What is the population of the German capital Berlin?"}, 
    {"output": "As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million."}
)

chat_with_memory = LLMChain(
    llm=chatGPT,
    prompt=prompt,
    verbose=False,
    memory=memory
)
pprint(chat_with_memory({"question":"How does it compare to the populatio of London?"}))

{'chat_history': 'Human: What is the population of the German capital Berlin?\n'
                 'AI: As of 2021, the estimated population of Berlin, the '
                 'capital city of Germany, is around 3.8 million.',
 'question': 'How does it compare to the populatio of London?',
 'text': 'As of 2021, the estimated population of London is around 8.9 '
         'million, making it significantly larger than Berlin.'}


In [84]:
pprint(chat_with_memory({"question":"How many languages are spoken in London?"}))

{'chat_history': 'Human: What is the population of the German capital Berlin?\n'
                 'AI: As of 2021, the estimated population of Berlin, the '
                 'capital city of Germany, is around 3.8 million.\n'
                 'Human: How does it compare to the populatio of London?\n'
                 'AI: As of 2021, the estimated population of London is around '
                 '8.9 million, making it significantly larger than Berlin.',
 'question': 'How many languages are spoken in London?',
 'text': 'According to the 2011 census, over 300 languages were spoken in '
         'London.'}


In [85]:
pprint(chat_with_memory({"question":"What is the population of the German capital Berlin as of 2023?"}))

{'chat_history': 'Human: What is the population of the German capital Berlin?\n'
                 'AI: As of 2021, the estimated population of Berlin, the '
                 'capital city of Germany, is around 3.8 million.\n'
                 'Human: How does it compare to the populatio of London?\n'
                 'AI: As of 2021, the estimated population of London is around '
                 '8.9 million, making it significantly larger than Berlin.\n'
                 'Human: How many languages are spoken in London?\n'
                 'AI: According to the 2011 census, over 300 languages were '
                 'spoken in London.',
 'question': 'What is the population of the German capital Berlin as of 2023?',
 'text': "I'm sorry, but I don't have access to real-time data. As of my last "
         'knowledge update in 2021, the estimated population of Berlin is '
         'around 3.8 million.'}


## Part 3. +  knowledge

In [86]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Chain for having a conversation based on retrieved documents.
from langchain.chains  import ConversationalRetrievalChain
import langchain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from pprint import pprint


info_about_berlin_loader = TextLoader("real-time-information-on-berlin.txt")

vectorstore = Chroma.from_documents(documents=info_about_berlin_loader.load(), 
                                    embedding=OpenAIEmbeddings())


chatGPT = ChatOpenAI()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_with_memory_and_knowledge = ConversationalRetrievalChain.from_llm(
    llm=chatGPT,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    memory=memory,
    # verbose=True
)

In [87]:
pprint(chat_with_memory_and_knowledge({"question":"What is the population of the German capital Berlin as of 2023?"}))

{'answer': 'The population of Berlin as of 2023 is 5,450,451.',
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin as of 2023?', additional_kwargs={}, example=False),
                  AIMessage(content='The population of Berlin as of 2023 is 5,450,451.', additional_kwargs={}, example=False)],
 'question': 'What is the population of the German capital Berlin as of 2023?'}


In [88]:
pprint(chat_with_memory_and_knowledge({"question":"How many of them are women?"}))

{'answer': 'As of 2023, the population of Berlin is 5,450,451 and 51% of that '
           'population are women. Therefore, the approximate number of women '
           'in the population of Berlin would be 2,777,727.',
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin as of 2023?', additional_kwargs={}, example=False),
                  AIMessage(content='The population of Berlin as of 2023 is 5,450,451.', additional_kwargs={}, example=False),
                  HumanMessage(content='How many of them are women?', additional_kwargs={}, example=False),
                  AIMessage(content='As of 2023, the population of Berlin is 5,450,451 and 51% of that population are women. Therefore, the approximate number of women in the population of Berlin would be 2,777,727.', additional_kwargs={}, example=False)],
 'question': 'How many of them are women?'}


In [91]:
langchain.debug = False
pprint(chat_with_memory_and_knowledge({"question":"How many languages are spoken in London?"}))

{'answer': "I don't know the exact number of languages spoken in London.",
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin as of 2023?', additional_kwargs={}, example=False),
                  AIMessage(content='The population of Berlin as of 2023 is 5,450,451.', additional_kwargs={}, example=False),
                  HumanMessage(content='How many of them are women?', additional_kwargs={}, example=False),
                  AIMessage(content='As of 2023, the population of Berlin is 5,450,451 and 51% of that population are women. Therefore, the approximate number of women in the population of Berlin would be 2,777,727.', additional_kwargs={}, example=False),
                  HumanMessage(content='How many languages are spoken in London?', additional_kwargs={}, example=False),
                  AIMessage(content="I don't know the exact number of languages spoken in London.", additional_kwargs={}, example=False),
                  HumanMessage(

## Part 4. +  knowledge & common knowledge 

In [92]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Chain for having a conversation based on retrieved documents.
from langchain.chains  import ConversationalRetrievalChain
import langchain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from pprint import pprint


info_about_berlin_loader = TextLoader("real-time-information-on-berlin.txt")

vectorstore = Chroma.from_documents(documents=info_about_berlin_loader.load(), 
                                    embedding=OpenAIEmbeddings())


chatGPT = ChatOpenAI()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

test_template = """System: Use the following pieces of context to answer the users question. \nIf you don't know the answer, answer the question once again ignoring the context.\n----------------\n{context}\nHuman: {question}"""
test_prompt = PromptTemplate(template=test_template, input_variables=["question", "context"])

chat_with_memory_and_knowledge = ConversationalRetrievalChain.from_llm(
    llm=chatGPT,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": test_prompt}
    # verbose=True
)

In [93]:
pprint(chat_with_memory_and_knowledge({"question":"What is the population of the German capital Berlin?"}))

{'answer': 'The population of the German capital Berlin is 5450451.',
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin?', additional_kwargs={}, example=False),
                  AIMessage(content='The population of the German capital Berlin is 5450451.', additional_kwargs={}, example=False)],
 'question': 'What is the population of the German capital Berlin?'}


In [96]:
pprint(chat_with_memory_and_knowledge({"question":"How many languages are spoken in London?"}))

{'answer': "As an AI language model, I don't have real-time data. However, as "
           'of my last available information, it is estimated that over 300 '
           'languages are spoken in London.',
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin?', additional_kwargs={}, example=False),
                  AIMessage(content='The population of the German capital Berlin is 5450451.', additional_kwargs={}, example=False),
                  HumanMessage(content='How many languages are spoken in London?', additional_kwargs={}, example=False),
                  AIMessage(content="As an AI language model, I don't have access to real-time data or the ability to browse the internet. However, London is a multicultural city with a diverse population, so it is likely that a large number of languages are spoken there.", additional_kwargs={}, example=False),
                  HumanMessage(content='How many languages are spoken in London?', additional_kw